from https://www.kaggle.com/code/imvision12/tensorflow-feedback-bert-baseline/notebook

# imports

In [ ]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from transformers import TFBertModel
import transformers


# inputs

In [ ]:
df = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
df.head()

In [ ]:
df['discourse_type'].unique()

# configuration

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 3
BATCH_SIZE = 16
MAX_LEN = 256

# Bert encoder

In [ ]:
def bert_encode(texts, tokenizer, max_len=MAX_LEN):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    
    for text in texts:
        token = tokenizer(text, max_length=max_len, truncation=True, padding='max_length',
                         add_special_tokens=True)
        input_ids.append(token['input_ids'])
        token_type_ids.append(token['token_type_ids'])
        attention_mask.append(token['attention_mask'])
    
    return np.array(input_ids), np.array(token_type_ids), np.array(attention_mask)

# loading bert tokenizer

In [ ]:

tokenizer = transformers.BertTokenizer.from_pretrained('../input/huggingface-bert-variants/bert-base-cased/bert-base-cased')
tokenizer.save_pretrained('.')

In [ ]:
sep = tokenizer.sep_token
sep

# adding discourse type to the input

In [ ]:
df['inputs'] = df.discourse_type + sep +df.discourse_text

In [ ]:
df.head()

# changing dicourse effectiveness label from text to label

In [ ]:
new_label = {"discourse_effectiveness": {"Ineffective": 0, "Adequate": 1, "Effective": 2}}
df = df.replace(new_label)
df = df.rename(columns = {"discourse_effectiveness": "label"})

In [ ]:
df.head()

# test train split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df['inputs'], df['label'], test_size=0.12, random_state=42)

# bert encoding train and test data 

since bert takes inputs in its own way

In [ ]:
X_train = bert_encode(X_train.astype(str), tokenizer)
X_valid = bert_encode(X_valid.astype(str), tokenizer)

y_train = y_train.values
y_valid = y_valid.values

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

# building model

In [ ]:
def build_model(bert_model, max_len=MAX_LEN):    
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = Dropout(.1)(clf_output)
    out = Dense(3, activation='softmax')(clf_output)
    
    model = Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
transformer_layer = (TFBertModel.from_pretrained('../input/huggingface-bert-variants/bert-base-cased/bert-base-cased'))
model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

# training

In [ ]:
train_history = model.fit(
    train_dataset,
    steps_per_epoch=200,
    validation_data=valid_dataset,
    epochs=5
)

# getting the text data

In [ ]:
test = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
test['text'] = test.discourse_type + sep +test.discourse_text
test.head()

# readying for test predictions

In [ ]:
test_text = bert_encode(test.text.astype(str), tokenizer)

In [ ]:
sub = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")
sub.head()

# prediction

In [ ]:
preds = model.predict(test_text, verbose=1)
preds

In [ ]:
sub['Ineffective'] = preds[:,0]
sub['Adequate'] = preds[:,1]
sub['Effective'] = preds[:,2]
sub

In [ ]:
sub.to_csv("submission.csv", index=False)